In [1]:
import pandas as pd
import numpy as np

import mlflow
import mlflow.sklearn

import mlflow
import dagshub

dagshub.init(repo_owner="abhijha8287", repo_name="beverage-price-prediction", mlflow=True)
mlflow.set_experiment("Beverage Price Prediction")


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report

from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

from xgboost import XGBClassifier

# =========================
# SET EXPERIMENT
# =========================
mlflow.set_experiment("Beverage Price Prediction")

# =========================
# LOAD DATA
# =========================
df = pd.read_csv("survey_results.csv")

# =========================
# DATA CLEANING
# =========================
df = df.drop_duplicates()
df = df[(df['age'] > 0) & (df['age'] <= 100)]

df['income_levels'] = df['income_levels'].fillna("Not Reported")
df['consume_frequency(weekly)'] = df['consume_frequency(weekly)'].fillna(df['consume_frequency(weekly)'].mode()[0])
df['purchase_channel'] = df['purchase_channel'].fillna(df['purchase_channel'].mode()[0])

for c in ['zone','current_brand','reasons_for_choosing_brands']:
    df[c] = df[c].astype(str).str.strip().str.title()

df['zone'] = df['zone'].replace({
    "Metor": "Metro",
    "Metrop": "Metro",
    "Semi Urban": "Semi-Urban",
    "Semiurban": "Semi-Urban"
})

df['current_brand'] = df['current_brand'].replace({
    "Establishd": "Established"
})

# =========================
# FEATURE ENGINEERING
# =========================

# age_group
bins = [0, 25, 35, 45, 55, 70, 200]
labels = ['18-25', '26-35', '36-45', '46-55', '56-70', '70+']
df['age_group'] = pd.cut(df['age'], bins=bins, labels=labels)
df = df.drop(columns=['age'])

# cf_ab_score
freq_map = {"0-2 times":1, "3-4 times":2, "5-7 times":3}
aware_map = {"0 to 1":1, "2 to 4":2, "above 4":3}

f = df['consume_frequency(weekly)'].map(freq_map)
a = df['awareness_of_other_brands'].map(aware_map)
df['cf_ab_score'] = (f / (f + a)).round(2)

# zas_score
zone_map = {"Urban":3, "Metro":4, "Rural":1, "Semi-Urban":2}
income_map = {"<10L":1, "10L - 15L":2, "16L - 25L":3, "26L - 35L":4, "> 35L":5, "Not Reported":0}

z = df['zone'].map(zone_map)
i = df['income_levels'].map(income_map)
df['zas_score'] = (z / (z + i)).round(2)

# BSI
df['bsi'] = ((df['current_brand'] != 'Established') & 
             (df['reasons_for_choosing_brands'].isin(['Price', 'Quality']))).astype(int)

# =========================
# REMOVE LOGICAL OUTLIERS
# =========================
df = df[~((df['occupation'] == 'Student') & (df['age_group'] == '56-70'))]

# =========================
# MODELING PREP
# =========================

y = df['price_range']
X = df.drop(columns=['respondent_id','price_range'])

# Label encoding
le_cols = ['age_group','income_levels','health_concerns','consume_frequency(weekly)','preferable_consumption_size']
le = LabelEncoder()
for col in le_cols:
    X[col] = le.fit_transform(X[col])

# One hot encoding
X = pd.get_dummies(X, drop_first=True)

# Encode target
y = LabelEncoder().fit_transform(y)

# Handle NaN
X = X.fillna(0)

# Train test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42
)

# =========================
# MODELS
# =========================
models = {
    "Gaussian Naive Bayes": GaussianNB(),
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "SVM": SVC(),
    "Random Forest": RandomForestClassifier(random_state=42),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)
}

results = {}

# =========================
# TRAIN + LOG TO MLFLOW
# =========================
for name, model in models.items():
    with mlflow.start_run(run_name=name):

        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        acc = accuracy_score(y_test, y_pred)

        # Log to MLflow
        mlflow.log_param("model_name", name)
        mlflow.log_metric("accuracy", acc)
        mlflow.sklearn.log_model(model, "model")

        results[name] = acc

        print("\n", "="*40)
        print(name, "Accuracy:", acc)
        print(classification_report(y_test, y_pred))

# =========================
# BEST MODEL
# =========================
best_model = max(results, key=results.get)
print("\n==============================")
print("BEST MODEL:", best_model)
print("==============================")


Accessing as abhijha8287

Initialized MLflow to track repo "abhijha8287/beverage-price-prediction"

Repository abhijha8287/beverage-price-prediction initialized!

2026/01/21 16:16:37 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
C:\Users\abhij\AppData\Roaming\Python\Python314\site-packages\langchain_core\_api\deprecation.py:26: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1



Gaussian Naive Bayes Accuracy: 0.5421284550674322
              precision    recall  f1-score   support

           0       0.46      0.22      0.30      1930
           1       0.52      0.28      0.37      2223
           2       0.63      0.90      0.74      2430
           3       0.43      0.91      0.59       906

    accuracy                           0.54      7489
   macro avg       0.51      0.58      0.50      7489
weighted avg       0.53      0.54      0.50      7489

🏃 View run Gaussian Naive Bayes at: https://dagshub.com/abhijha8287/beverage-price-prediction.mlflow/#/experiments/0/runs/4b68972c5da844d8b7be4c9fa31306d5
🧪 View experiment at: https://dagshub.com/abhijha8287/beverage-price-prediction.mlflow/#/experiments/0


...


SVM Accuracy: 0.8615302443583923
              precision    recall  f1-score   support

           0       0.82      0.84      0.83      1930
           1       0.82      0.85      0.83      2223
           2       0.93      0.91      0.92      2430
           3       0.87      0.81      0.84       906

    accuracy                           0.86      7489
   macro avg       0.86      0.85      0.86      7489
weighted avg       0.86      0.86      0.86      7489

🏃 View run SVM at: https://dagshub.com/abhijha8287/beverage-price-prediction.mlflow/#/experiments/0/runs/a7e2213587374158bf1189f40b5e5180
🧪 View experiment at: https://dagshub.com/abhijha8287/beverage-price-prediction.mlflow/#/experiments/0


2026/01/21 16:18:44 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



Random Forest Accuracy: 0.8935772466283883
              precision    recall  f1-score   support

           0       0.89      0.86      0.87      1930
           1       0.85      0.89      0.87      2223
           2       0.94      0.93      0.93      2430
           3       0.91      0.87      0.89       906

    accuracy                           0.89      7489
   macro avg       0.90      0.89      0.89      7489
weighted avg       0.89      0.89      0.89      7489

🏃 View run Random Forest at: https://dagshub.com/abhijha8287/beverage-price-prediction.mlflow/#/experiments/0/runs/3a4a70b1f587478aac1ca4248b39247a
🧪 View experiment at: https://dagshub.com/abhijha8287/beverage-price-prediction.mlflow/#/experiments/0


C:\Users\abhij\AppData\Roaming\Python\Python314\site-packages\xgboost\training.py:199: UserWarning: [16:20:33] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
2026/01/21 16:20:35 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



XGBoost Accuracy: 0.9202830818533849
              precision    recall  f1-score   support

           0       0.92      0.90      0.91      1930
           1       0.89      0.91      0.90      2223
           2       0.95      0.95      0.95      2430
           3       0.92      0.91      0.92       906

    accuracy                           0.92      7489
   macro avg       0.92      0.92      0.92      7489
weighted avg       0.92      0.92      0.92      7489

🏃 View run XGBoost at: https://dagshub.com/abhijha8287/beverage-price-prediction.mlflow/#/experiments/0/runs/dfb1c7ae719e4d62916562a9cccd6806
🧪 View experiment at: https://dagshub.com/abhijha8287/beverage-price-prediction.mlflow/#/experiments/0

BEST MODEL: XGBoost


In [2]:
import joblib

# Save best model
joblib.dump(models[best_model], "best_model.pkl")

# Save training columns (VERY IMPORTANT for Streamlit)
joblib.dump(X.columns.tolist(), "model_columns.pkl")

print("Model and columns saved successfully!")


Model and columns saved successfully!
